# Data Wrangling Exercises

## Zillow

### Acquire and Summarize

#### 1. 
Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [1]:
import os
from env import user, password, host
import pandas as pd
# Fix so our 'check_columns will display correctly
pd.options.display.max_rows = 69


def get_zillow(user=user,password=password,host=host):
    """
    This function acquires data from a SQL database of 2017 Zillow properties and caches it locally.
    
    :param user: The username for accessing the MySQL database
    :param password: The password is unique per user saved in env
    :param host: The host parameter is the address of the server where the Zillow database is hosted
    :return: The function `get_zillow` is returning a dirty pandas DataFrame
    containing information on 2017 Zillow properties
    """
    # name of cached csv
    filename = 'zillow.csv'
    # if cached data exist
    if os.path.isfile(filename):
        df = pd.read_csv(filename, dtype={'buildingclassdesc':'string'})
    # wrangle from sql db if not cached
    else:
        # read sql query into df
        df = pd.read_sql('''
                        select *
                        from properties_2017
                        left join airconditioningtype using(airconditioningtypeid)
                        left join architecturalstyletype using(architecturalstyletypeid)
                        left join buildingclasstype using(buildingclasstypeid)
                        left join heatingorsystemtype using(heatingorsystemtypeid)
                        left join predictions_2017 using(parcelid)
                        left join propertylandusetype using(propertylandusetypeid)
                        left join storytype using(storytypeid)
                        left join typeconstructiontype using(typeconstructiontypeid)
                        where transactiondate like %s
                        '''
                        , f'mysql+pymysql://{user}:{password}@{host}/zillow'
                        , params=[('2017%',)]
                        )
        # cache data locally
        df.to_csv(filename, index=False)
    # sort by column: 'transactiondate' (descending) for dropping dupes keeping recent
    df = df.sort_values(['transactiondate'], ascending=[False]).drop_duplicates(subset=['parcelid']).sort_index()
    # no null lat long
    df = df[df['latitude'].notna()]
    print(f"Total rows: {df.shape[0]}")
    print(f"Total columns: {df.shape[1]}")
    return df

df = get_zillow()

Total rows: 77381
Total columns: 69


#### 2. 

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77381 entries, 0 to 77612
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77381 non-null  float64
 3   parcelid                      77381 non-null  int64  
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  calcul

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77381.0,2.618263e+02,5.141199e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
parcelid,77381.0,1.300715e+07,3.481346e+06,1.071186e+07,1.153830e+07,1.253157e+07,1.421183e+07,1.676893e+08
heatingorsystemtypeid,49440.0,3.920510e+00,3.592779e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,77381.0,1.495139e+06,8.609071e+05,3.490000e+02,7.520700e+05,1.497932e+06,2.240535e+06,2.982274e+06
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03


In [4]:
def check_columns(df):
    """
    This function takes a pandas dataframe as input and returns
    a dataframe with information about each column in the dataframe. For
    each column, it returns the column name, the number of
    unique values in the column, the unique values themselves,
    the number of null values in the column, the proportion of null values,
    the data type of the column, and the range of the column if it is float or int. The resulting dataframe is sorted by the
    'Number of Unique Values' column in ascending order.

    Args:
    - df: pandas dataframe

    Returns:
    - pandas dataframe
    """
    print(f"Total rows: {df.shape[0]}")
    print(f"Total columns: {df.shape[1]}")
    data = []
    # Loop through each column in the dataframe
    for column in df.columns:
        # Append the column name, number of unique values, unique values, number of null values, proportion of null values, and data type to the data list
        if df[column].dtype in ['float64', 'int64']:
            data.append(
                [
                    column,
                    df[column].dtype,
                    df[column].nunique(),
                    df[column].isna().sum(),
                    df[column].isna().mean().round(5),
                    df[column].unique(),
                    df[column].describe()[['min', 'max', 'mean']].values
                ]
            )
        else:
            data.append(
                [
                    column,
                    df[column].dtype,
                    df[column].nunique(),
                    df[column].isna().sum(),
                    df[column].isna().mean().round(5),
                    df[column].unique(),
                    None
                ]
            )
    # Create a pandas dataframe from the data list, with column names 'Column Name', 'Number of Unique Values', 'Unique Values', 'Number of Null Values', 'Proportion of Null Values', 'dtype', and 'Range' (if column is float or int)
    # Sort the resulting dataframe by the 'Number of Unique Values' column in ascending order
    return pd.DataFrame(
        data,
        columns=[
            "col_name",
            "dtype",
            "num_unique",
            "num_null",
            "pct_null",
            "unique_values",
            "range (min, max, mean)"
        ])


In [5]:
check_columns(df)

Total rows: 77381
Total columns: 69


,col_name,dtype,num_unique,num_null,pct_null,unique_values,"range (min, max, mean)"
0,typeconstructiontypeid,float64,4,77159,0.99713,"[nan, 6.0, 13.0, 10.0, 4.0]","[4.0, 13.0, 6.04054054054054]"
1,storytypeid,float64,1,77331,0.99935,"[nan, 7.0]","[7.0, 7.0, 7.0]"
2,propertylandusetypeid,float64,13,0,0.00000,"[261.0, 266.0, 246.0, 260.0, 269.0, 247.0, 248...","[31.0, 275.0, 261.8263397991755]"
3,parcelid,int64,77381,0,0.00000,"[14297519, 17052889, 14186244, 12177905, 10887...","[10711855.0, 167689317.0, 13007150.505731381]"
4,heatingorsystemtypeid,float64,10,27941,0.36108,"[nan, 2.0, 7.0, 6.0, 24.0, 13.0, 18.0, 20.0, 1...","[1.0, 24.0, 3.920509708737864]"
5,buildingclasstypeid,float64,2,77366,0.99981,"[nan, 4.0, 3.0]","[3.0, 4.0, 3.933333333333333]"
6,architecturalstyletypeid,float64,5,77175,0.99734,"[nan, 7.0, 8.0, 21.0, 3.0, 2.0]","[2.0, 21.0, 7.388349514563107]"
7,airconditioningtypeid,float64,5,52428,0.67753,"[nan, 1.0, 13.0, 5.0, 11.0, 9.0]","[1.0, 13.0, 1.8132889832885826]"
8,id,int64,77381,0,0.00000,"[1727539, 1387261, 11677, 2288172, 1970746, 14...","[349.0, 2982274.0, 1495138.9017588296]"
9,basementsqft,float64,43,77331,0.99935,"[nan, 900.0, 1416.0, 224.0, 100.0, 512.0, 380....","[38.0, 3560.0, 679.72]"


#### 3. 

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

##### The output should look like the table below:


|  | num_rows_missing | pct_rows_missing|
| - |------------------|-----------------|
| parcelid | 0 | 0.000000|
| airconditioningtypeid | 29041 | 0.535486|
| architecturalstyletypeid | 54232 | 0.999982 |

In [6]:
def check_columns_only_nulls(df):
    """
    This function takes a pandas dataframe as input and returns
    a dataframe with information about each column in the dataframe. For
    each column, it returns the column name, the number of
    unique values in the column, the unique values themselves,
    the number of null values in the column, the proportion of null values,
    and the data type of the column. The resulting dataframe is sorted by the
    'Number of Unique Values' column in ascending order.

    Args:
    - df: pandas dataframe

    Returns:
    - pandas dataframe
    """
    data = []
    # Loop through each column in the dataframe
    for column in df.columns:
        # Append the column name, number of unique values, unique values, number of null values, proportion of null values, and data type to the data list
        data.append(
            [
                column,
                # df[column].nunique(),
                # df[column].unique(),
                df[column].isna().sum(),
                df[column].isna().mean(),
                # df[column].dtype,
            ]
        )
    # Create a pandas dataframe from the data list, with column names 'Column Name', 'Number of Unique Values', 'Unique Values', 'Number of Null Values', 'Proportion of Null Values', and 'dtype'
    # Sort the resulting dataframe by the 'Number of Unique Values' column in ascending order
    return pd.DataFrame(
        data,
        columns=[
            "column_name",
            # "Number of Unique Values",
            # "Unique Values",
            "num_rows_missing",
            "pct_rows_missing",
            # "dtype",
        ],
    )

In [7]:
check_columns_only_nulls(df)

,column_name,num_rows_missing,pct_rows_missing
0,typeconstructiontypeid,77159,0.997131
1,storytypeid,77331,0.999354
2,propertylandusetypeid,0,0.000000
3,parcelid,0,0.000000
4,heatingorsystemtypeid,27941,0.361083
5,buildingclasstypeid,77366,0.999806
6,architecturalstyletypeid,77175,0.997338
7,airconditioningtypeid,52428,0.677531
8,id,0,0.000000
9,basementsqft,77331,0.999354


### Prepare

#### 1. 

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.



#### 2. 

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).


**The input:**
- A dataframe

- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).

- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

**The output:**

- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

- (Hint) Look up the dropna documentation.

- You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [8]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    """
    Drop rows or columns based on the percent of values that are missing.

    Parameters:
    df (pandas.DataFrame): The input dataframe.
    prop_required_column (float): The proportion of non-missing values required to keep a column.
    prop_required_row (float): The proportion of non-missing values required to keep a row.

    Returns:
    pandas.DataFrame: The dataframe with missing values handled.
    """
    temp_df = df
    # Drop columns with too many missing values
    threshold = int(round(prop_required_column * len(df.index), 0))
    temp_df.dropna(axis=1, thresh=threshold, inplace=True)

    # Drop rows with too many missing values
    threshold = int(round(prop_required_row * len(df.columns), 0))
    temp_df.dropna(axis=0, thresh=threshold, inplace=True)

    return temp_df

In [9]:
df1 = handle_missing_values(df, 0, 0)

check_columns(df1)

#### 3. 

Encapsulate your work inside of functions in a wrangle_zillow.py module.

## Mall Customers

### 1. 
Acquire data from the customers table in the mall_customers database.

### 2. 
Summarize the data (include distributions and descriptive statistics).

### 3. 
Detect outliers using IQR.


### 4. 
Split data into train, validate, and test.


### 5. 
Encode categorical columns using a one hot encoder (pd.get_dummies).


### 6. 
Handles missing values.


### 7. 
Scaling



### 8. 
Encapsulate your work in a `wrangle_mall.py` python module.